## Import stuff

In [22]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time
import sklearn.metrics as metrics


## Load training data from csv


In [23]:
val_data_1 = pd.read_csv("val_distances_dropout.csv",index_col=0)
val_data_2 = pd.read_csv("val_distances_batchnorm.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)

for extractor in ["xl","resnet","clip","vit","swin"]:
    val_data_1[extractor] = val_data_1["anchor_left_distance_"+extractor]-val_data_1["anchor_right_distance_"+extractor]
    val_data_1.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
    val_data_1.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


for extractor in ["xl","resnet","clip","vit","swin"]:
    val_data_2[extractor] = val_data_2["anchor_left_distance_"+extractor]-val_data_2["anchor_right_distance_"+extractor]
    val_data_2.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
    val_data_2.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


X_val_1 = np.array(val_data_1.iloc[:, :].values,  dtype=float)
X_val_2 = np.array(val_data_2.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_val_1.shape)
print("X shape: ", X_val_2.shape)
print("Y shape: ", Y_val.shape)

val_data_2

X shape:  (7471, 5)
X shape:  (7471, 5)
Y shape:  (7471,)


,xl,resnet,clip,vit,swin
0,-0.379850,-0.557025,-0.497801,-0.276974,-0.556594
1,-0.138518,-0.102112,-0.203814,-0.125433,0.020896
2,0.105196,0.105294,-0.078201,-0.019630,0.043718
3,-0.734423,-0.804009,-1.093797,-1.136176,-0.732954
4,0.595210,0.409195,0.475904,-0.140896,0.330896
...,...,...,...,...,...
7466,-0.428260,-0.201487,-0.830325,-0.656267,-0.522852
7467,0.053856,0.292658,0.027265,0.154834,-0.199139
7468,-0.221421,-0.591940,0.112721,-0.294572,-0.326937
7469,0.468808,0.536336,0.491642,0.217314,0.357579


# weighted ensemble
the weights would be kinda hard to estimate in practice, it would take a few server submissions. I just made the worse 2 models less important

In [37]:
pred = []
for (row1,row2) in zip(X_val_1,X_val_2):
    score_left = 1*(0.5*row1[0]+row1[1]+row1[2]+0.5*row1[3]) + 1*(0.5*row2[0]+row2[1]+row2[2]+0.5*row2[3])
    pred.append(score_left>0)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred)
accuracy

0.7832954089144692

# Majority vote
performs worse

In [25]:
# pred = []
# for row in X_val:
#     score_left = 0
#     score_left += row[0]>0
#     score_left += row[1]>0
#     score_left += row[2]>0
#     score_left += row[3]>0
#     score_left += row[4]>0
#     pred.append(0 if score_left>2 else 1)

# pred = np.array(pred)
# accuracy = metrics.accuracy_score(Y_val, pred<0.5)
# accuracy

# Miss-classified triplets

In [26]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 3, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

val_triplets = np.loadtxt('./val_triplets_split.txt', dtype=str,delimiter = ' ')

In [27]:
id=85

In [28]:
while True:
    #find next triplet that was missclassified, and plot it
    id+=1
    if pred[id] != Y_val[id]:
        anchor = Image.open('food_224/' + val_triplets[id][0] + '.jpg')
        left = Image.open('food_224/' + val_triplets[id][1] + '.jpg')
        right = Image.open('food_224/' + val_triplets[id][2] + '.jpg')
        row = X_val[id]
        score_left = 0.5*row[0]+row[1]+row[2]+0.5*row[3]
        print("id: " + str(id) + " model predicted: " + str(score_left) + " label: " + ("left" if Y_val[id]==0 else "right"))
        plotImages([anchor, left, right])
        break
        


NameError: name 'X_val' is not defined